In [1]:
import pandas as pd

In [31]:
year = int(input('Enter the year: '))
month = int(input('Enter the month from january till september (1-11) if you dont want to filter by month insert, 0: ')) 


KeyboardInterrupt: 

### Acquisition

In [9]:
def acquire():
    file = "./data/tenis_{}.csv".format(year)
    df = pd.read_csv(file)
    return df 

In [10]:
data = acquire()

### Wrangling

In [11]:
def wrangle(data): 
    data["tourney_date"] = pd.to_datetime(data.tourney_date.astype("str")) # le doy formato de fecha a tourney date
    data["tourney_month"] = data["tourney_date"].dt.month # creo una columna con el mes que figura en tourney date
    if month == 0:
        filtered = data
    else: 
        filtered = data[data["tourney_month"] == month] # Filtro el dataframe por el mes que me interesa
    return filtered

In [12]:
filtered = wrangle(data)

###  Analyze

In [59]:
#def analize(filtered):
grouped = filtered["winner_name"].value_counts().head(10) # cantidad de partidos ganados en el mes 

""" media de partidos ganados al mes por jugador"""
monthly_mean = round(data.groupby("winner_name")["winner_name"].count()/(len(set(data['tourney_month']))),1) 



In [111]:

# convierto las series en Dataframes para poderles hacer un merge eventualmente y ponerlas en la misma tabla. 
pd_grouped = (pd.DataFrame(grouped)).reset_index(level=0)
pd_grouped = pd_grouped.rename(columns={"winner_name":"winned matches","index":"winner_name"})
df_mm = pd.DataFrame(monthly_mean)
df_mm.index.names = ["winner"]
df_mm = (df_mm.reset_index(level=0))
df_mm = df_mm.rename(columns={"winner_name":"avg winned matches per month","winner":"winner_name"})
df_merge = pd.merge(pd_grouped,df_mm,on="winner_name")


,winner_name,winned matches,avg winned matches per month
0,Rafael Nadal,67,6.1
1,David Goffin,59,5.4
2,Alexander Zverev,55,5.0
3,Roger Federer,53,4.8
4,Grigor Dimitrov,49,4.5
5,Roberto Bautista Agut,48,4.4
6,Dominic Thiem,47,4.3
7,Marin Cilic,45,4.1
8,Jack Sock,39,3.5
9,Diego Sebastian Schwartzman,39,3.5


In [109]:

pd_grouped

,winner_name,winned matches
0,Rafael Nadal,67
1,David Goffin,59
2,Alexander Zverev,55
3,Roger Federer,53
4,Grigor Dimitrov,49
5,Roberto Bautista Agut,48
6,Dominic Thiem,47
7,Marin Cilic,45
8,Jack Sock,39
9,Diego Sebastian Schwartzman,39
